In [9]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))
from utils import *

# The idea

First, retrieve all scene objects' configuration and position. 

In [10]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [11]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [13]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [14]:
listazinha

[{'object_name': 'Cuboid_0',
  'object_handler': 16,
  'object_position': [-1.7249997854232788,
   0.25000035762786865,
   0.2500000596046448],
  'object_orientation': [3.993277264413564e-09,
   -1.0491065655093834e-10,
   5.78318428357223e-17]},
 {'object_name': 'Cuboid_1',
  'object_handler': 17,
  'object_position': [-0.3500001132488251, 1.4999994039535522, 0.5],
  'object_orientation': [-1.2080765277033834e-09,
   -8.589069039333097e-11,
   6.892211546491792e-14]},
 {'object_name': 'Cuboid_2',
  'object_handler': 18,
  'object_position': [-1.5249994993209839, -1.2249995470046997, 0.5],
  'object_orientation': [-1.0236179681655244e-09,
   -1.39426714618196e-09,
   3.488473429531314e-17]},
 {'object_name': 'Cuboid_3',
  'object_handler': 19,
  'object_position': [-0.1499994993209839,
   -0.09999959170818329,
   0.2500000596046448],
  'object_orientation': [1.1374869934854814e-09,
   -1.3272767607208635e-10,
   1.5923003452880393e-17]},
 {'object_name': 'Cuboid_4',
  'object_handler':